#task
# CPO доволен тем как вы справились с RFM анализом. Ваша новая задача сделать когортный анализ.

Что такое когортный анализ?
Когортный анализ заключается в исследовании характеристик когорт/винтажей/поколений, объединенных по общим временным признакам.
Когорта/винтаж/поколение — это группа, сформированная особым образом по временному признаку: например, месяцу регистрации, месяцу первой транзакции или первого посещения сайта. Когорты очень похожи на сегменты с тем отличием, что когорта объединяет группы определенного промежутка времени, в то время как сегмент может быть основан на любых других характеристиках.

Почему это ценно?
Когортный анализ может быть полезен, когда речь заходит о понимании здоровья вашего бизнеса и «липкости» - лояльности ваших клиентов. «Липкость» имеет решающее значение, так как гораздо дешевле и проще удерживать клиента, чем приобретать новых. Кроме того, ваш продукт развивается с течением времени. Новые функции добавляются и удаляются, измененяется дизайн и т. д. Наблюдение отдельных групп с течением времени является отправной точкой для понимания того, как эти изменения влияют на поведение пользователя/группы.

Для этой задачи так же используйте публичный датасет: https://www.kaggle.com/olistbr/brazilian-ecommerce Файлы olist_orders_dataset.csv и olist_order_payments_dataset.csv. Соединить их можно по order_id.

Ответьте на два вопроса:
- Сколько в среднем заказов и на какую сумму делают все когорты за первый год
- Сравните две любые когорты по прибыле и количеству заказов

Подсказки по коду:
Для когорт будем использовать order_delivered_carrier_date
first_orders = order_payment.groupby('customer_id')['order_delivered_carrier_date'].agg({'first_order': 'min'})

Полезный материал для изучения:
- https://gopractice.ru/cohort_analysis/
- https://habr.com/ru/company/io/blog/262025/
- https://www.owox.ru/blog/use-cases/cohort-analysis/

In [2]:
import pandas as pd

C:\Users\Nursultan\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
orders = pd.read_csv('../data/task1/olist_orders_dataset.csv')
payments = pd.read_csv('../data/task1/olist_order_payments_dataset.csv')

In [5]:
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])


In [6]:
df = pd.merge(orders, payments, on='order_id')


In [7]:
first_orders = df.groupby('customer_id')['order_delivered_carrier_date'].min().reset_index()

In [9]:
first_orders.columns = ['customer_id', 'first_order']

In [10]:
df = pd.merge(df, first_orders, on='customer_id')

In [15]:
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value,first_order,order_month,cohort
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,credit_card,1,18.12,2017-10-04 19:55:00,2017-10,2017-10
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3,voucher,1,2.00,2017-10-04 19:55:00,2017-10,2017-10
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2,voucher,1,18.59,2017-10-04 19:55:00,2017-10,2017-10
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,boleto,1,141.46,2018-07-26 14:31:00,2018-07,2018-07
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,credit_card,3,179.12,2018-08-08 13:50:00,2018-08,2018-08


In [11]:
# Получим когорту для каждого заказа (месяц и год первого заказа клиента)
df['order_month'] = df['order_delivered_carrier_date'].dt.to_period('M')
df['cohort'] = df['first_order'].dt.to_period('M')

Теперь, когда у нас есть когорта для каждого заказа, мы можем агрегировать данные по когортам и месяцам.

In [12]:
# Считаем количество заказов и общую сумму по месяцам и когортам
cohort_data = df.groupby(['cohort', 'order_month']).agg({'payment_value': ['sum', 'count']}).reset_index()

In [13]:
# Ограничиваем данные первым годом деятельности каждой когорты
one_year_cohorts = cohort_data[cohort_data['order_month'] <= (cohort_data['cohort'] + 12)]

# Считаем среднее количество заказов и среднюю сумму заказов за первый год
average_orders = one_year_cohorts[('payment_value', 'count')].mean()
average_revenue = one_year_cohorts[('payment_value', 'sum')].mean()

print('Среднее количество заказов на когорту за первый год:', average_orders)
print('Средняя выручка на когорту за первый год:', average_revenue)

Среднее количество заказов на когорту за первый год: 4249.916666666667
Средняя выручка на когорту за первый год: 650339.80875


In [14]:
# Получаем данные по первой и второй когорте
cohort1 = cohort_data[cohort_data['cohort'] == '2017-01']
cohort2 = cohort_data[cohort_data['cohort'] == '2017-02']

# Считаем общее количество заказов и общую выручку для каждой когорты
total_orders_cohort1 = cohort1[('payment_value', 'count')].sum()
total_revenue_cohort1 = cohort1[('payment_value', 'sum')].sum()

total_orders_cohort2 = cohort2[('payment_value', 'count')].sum()
total_revenue_cohort2 = cohort2[('payment_value', 'sum')].sum()

print('Когорта 1 - Заказы:', total_orders_cohort1, 'Выручка:', total_revenue_cohort1)
print('Когорта 2 - Заказы:', total_orders_cohort2, 'Выручка:', total_revenue_cohort2)

Когорта 1 - Заказы: 656 Выручка: 97060.76
Когорта 2 - Заказы: 1594 Выручка: 258761.05
